In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu113/torch-1.11.0%2Bcu113-cp310-cp310-linux_x86_64.whl (1637.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1
    Uninstalling torch-2.2.1:
      Successfully uninstalled torch-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
curated-transformers 0.1.1 requires torch>=1.12.0, but you have torch 1.11.0+cu113 which is incompatible.
spacy-curated-transformers 0.2.2 requires torch>=1.12.0, but you have torch 1.11.0+cu113 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0+cu113 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0+cu113 which is incompatible.


In [ ]:
!export CUDA_PATH="/opt/nvidia/cuda"
!pip install -U spacy[cuda113,transformers]

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.1 MB/s eta 0:00:00
  Using cached torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 2.2.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.2.1 which is incompatible.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 2.2.1 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies

In [ ]:
import spacy
from thinc.api import set_gpu_allocator, require_gpu

In [ ]:
!pip install cupy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: Could not build wheels for cupy, which is required to install pyproject.toml-based projects


In [ ]:
import pandas as pd
import os
from collections import Counter
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Trailadata_preprocessed.csv', encoding = "UTF-8")
df.head()

,File Name,Transcript,Label,tokenized_Transcript,clean_text,wordcount
0,trial_truth_002.txt,"All of us, who have represented people for yea...",truthful,all of us who have represented people for year...,represent people year system letter prisioner ...,57
1,trial_truth_001.txt,Reforming the criminal justice system matters ...,truthful,reforming the criminal justice system matters ...,reform criminal justice system matter law legi...,36
2,trial_truth_006.txt,They really didn't tell me anything. They said...,truthful,they really did n't tell me anything they said...,tell say accident say accident ian fine accide...,70
3,trial_truth_004.txt,"I do. I was, uh... in the office, and I got a ...",truthful,i do i was uh in the office and i got a call u...,uh office get uh maybe close o clock remember ...,221
4,trial_truth_003.txt,"But yes, I was there. Yep, I stayed. Uh ... Ye...",truthful,but yes i was there yep i stayed uh yep prob y...,yes yep stay uh yep prob yes yes,13


In [ ]:
# Using spacy.load().
nlp = spacy.load("en_core_web_trf", disable = ["tagger", "parser", "attribute_ruler", "lemmatizer"])

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
Trans_txt= []

for txt in nlp.pipe(df['Label']):
    Trans_txt.append(txt)

In [ ]:
df['wordcount'] = pd.Series(list(df['wordcount']))

In [ ]:
print(df['wordcount'].dtype)

int64


In [ ]:
# Create an empty list to store the Entities
entity_store = []

In [ ]:
# Initialize an empty list to store named entity counts for all documents
entity_store = []

# Loop through each document
for txt, label in zip(Trans_txt, df['Label']):
    # Initialize a dictionary to store named entity counts for the current document
    txt_entity_counts = {}

    # Loop through each named entity in the current document
    for entity in txt.ents:
        # Get the text and label of the named entity
        entity_text, entity_label = entity.text, entity.label_

        # Update the counts for the current entity label
        if entity_label not in txt_entity_counts:
            txt_entity_counts[entity_label] = 1  # Initialize count for new entity label
        else:
            txt_entity_counts[entity_label] += 1  # Increment count for existing entity label

    # Append the named entity counts for the current document to the list
    txt_entity_counts['Label'] = label  # Add the label for the current document
    entity_store.append(txt_entity_counts)

# Create a DataFrame from the list of dictionaries with named entity counts
nameentityrec_df = pd.DataFrame.from_records(entity_store).fillna(0)

# Normalize the counts by dividing by the total word counts and multiplying by 100
nameentityrec_df = nameentityrec_df.div(df['wordcount'], axis=0) * 100

# Display the resulting DataFrame
nameentityrec_df


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
print(df['wordcount'].dtype)
print(nameentityrec_df['Label'].dtype)

int64
object


In [ ]:
print(df.head())
print(nameentityrec_df.head())

             File Name                                         Transcript  \
0  trial_truth_002.txt  All of us, who have represented people for yea...   
1  trial_truth_001.txt  Reforming the criminal justice system matters ...   
2  trial_truth_006.txt  They really didn't tell me anything. They said...   
3  trial_truth_004.txt  I do. I was, uh... in the office, and I got a ...   
4  trial_truth_003.txt  But yes, I was there. Yep, I stayed. Uh ... Ye...   

      Label                               tokenized_Transcript  \
0  truthful  all of us who have represented people for year...   
1  truthful  reforming the criminal justice system matters ...   
2  truthful  they really did n't tell me anything they said...   
3  truthful  i do i was uh in the office and i got a call u...   
4  truthful  but yes i was there yep i stayed uh yep prob y...   

                                          clean_text  wordcount  
0  represent people year system letter prisioner ...         57  
1  refor